Importing libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Preparing dataset

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/data_storm_2/new-set.csv')

In [ ]:
y = dataset.iloc[:,20].values
cols = [0,20]   #drop id and checkin
dataset.drop(dataset.columns[cols],axis=1,inplace=True)
x = dataset.iloc[:,:].values

In [ ]:
y = np.where(y=="Check-In",1,y)     #encode
y = np.where(y=="Canceled",2,y)
y = np.where(y=="No-Show",3,y)

*Preprocessing - Date Conversion*

In [ ]:
booking = x[:,9]       #copy booking dates
checkout = x[:,8]      #copy check-in dates
checkin = x[:,7]       #copy check-out dates

In [ ]:
### func and class template to calculate the date difference between two given calendar dates
### source : w3schools | added minor modifications

class Date:
    def __init__(self, d, m, y):
        self.d = d
        self.m = m
        self.y = y
 
monthDays = [31, 28, 31, 30, 31, 30,
             31, 31, 30, 31, 30, 31]
 
def countLeapYears(d):
    years = d.y
    if (d.m <= 2):
        years -= 1
    return int(years / 4) - int(years / 100) + int(years / 400)
 
def getDifference(dt1, dt2): 
    n1 = dt1.y * 365 + dt1.d
    for i in range(0, dt1.m - 1):
        n1 += monthDays[i]
    n1 += countLeapYears(dt1)
    n2 = dt2.y * 365 + dt2.d
    for i in range(0, dt2.m - 1):
        n2 += monthDays[i]
    n2 += countLeapYears(dt2)
    return (n2 - n1)

In [ ]:
diff1 = [] #difference in days between booking and checkin
diff2 = [] #difference in days between checkin and checkout
month = [] #month of each booking to capture any seasonal variations

for i in range(len(booking)):
  arr1 = booking[i].split("/")  #booking date
  arr2 = checkin[i].split("/")  #checkin date
  arr3 = checkout[i].split("/")  #checkout date
  month.append(int(arr2[0]))   #stores month of each booking to capture any seasonal variations
  dt1 = Date(int(arr1[1]), int(arr1[0]), int(arr1[2]))   #booking date
  dt2 = Date(int(arr2[1]), int(arr2[0]), int(arr2[2]))   #checkin date
  dt3 = Date(int(arr3[1]), int(arr3[0]), int(arr3[2]))   #checkout date
  diff1.append(getDifference(dt1, dt2))   #difference in days between booking and checkin
  diff2.append(getDifference(dt2, dt3))   #difference in days between checkin and checkout


In [ ]:
np_month = np.array(month).reshape(len(booking),1)   #reshaping np obj
np_diff1 = np.array(diff1).reshape(len(booking),1)   #reshaping np obj
np_diff2 = np.array(diff2).reshape(len(booking),1)   #reshaping np obj

In [ ]:
x = np.append(x, np_month, axis=1)    #append new column | month -> captures any seasonal variations that might affect bookings
x = np.append(x, np_diff1, axis=1)    #append new column | difference in days between booking and checkin -> how well in advance does the user books
x = np.append(x, np_diff2, axis=1)    #append new column | difference in days between checkin and checkout -> duration of stay planned by the user

In [ ]:
x = np.delete(x,7,axis=1)       #delete checkin date column
x = np.delete(x,7,axis=1)       #delete checkout date column
x = np.delete(x,7,axis=1)       #delete booking date column

Preprocessing - One Hot Encoding

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [ ]:
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(), [0,2,3,4,5,6,10,11,12,13,14,15,16,19])], remainder='passthrough')

In [ ]:
x = ct.fit_transform(x)     #apply one hot label encoding to categorical data 

Splitting the dataset

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.0001, random_state = 1)

Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()                             #apply feature scaling to remove bias from scalar values

In [ ]:
x_train[:, 50:] = sc.fit_transform(x_train[:, 50:])    #ignore the label encoded columns
x_test[:, 50:] = sc.transform(x_test[:, 50:])          #use the same scaler

# **Classification model**

Parameter Tuning

In [ ]:
# #parameter tuning
# from sklearn.model_selection import GridSearchCV 
  
# defining parameter range 
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
grid.fit(x_train.astype(int), y_train.astype(int)) 

# print best parameter after tuning 
print(grid.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid.best_estimator_) 

Check on validation set

In [ ]:
## add code here to test on validation

Training the model

In [ ]:
from lightgbm import LGBMClassifier
classifier = LGBMClassifier()
classifier.fit(x_train.astype(int), y_train.astype(int))

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
# # print best parameter after tuning 
# print(grid.best_params_) 
  
# # print how our model looks after hyper-parameter tuning 
# print(grid.best_estimator_) 

Prediction on train data set

In [ ]:
y_pred = classifier.predict(x_test)     #predict on the test set of the training data

Confusion Matrix

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test.astype(int), y_pred.astype(int)))
print(classification_report(y_test.astype(int), y_pred.astype(int)))

[[4]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         4

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4



## Test on the validation set / new dataset

In [ ]:
#test set
#testset = pd.read_csv('/content/drive/MyDrive/data_storm_2/Hotel-A-test.csv') 
#validation set
testset = pd.read_csv('/content/drive/MyDrive/data_storm_2/Hotel-A-validation.csv')      

In [ ]:
z = testset.iloc[:,1:].values        #remove first column

In [ ]:
booking = z[:,9]           #copy bookin dates
checkout = z[:,8]          #copy checkout dates
checkin = z[:,7]           #copy checkin dates

In [ ]:
diff1 = [] #stores difference in days between booking and checkin
diff2 = [] #stores difference in days between checkin and checkout
month = [] #stores month of each booking to capture any seasonal variations

for i in range(len(booking)):
  arr1 = booking[i].split("/")  #booking date
  arr2 = checkin[i].split("/")  #checkin date
  arr3 = checkout[i].split("/")  #checkout date
  month.append(int(arr2[0]))   #stores month of each booking to capture any seasonal variations
  dt1 = Date(int(arr1[1]), int(arr1[0]), int(arr1[2]))   #booking date
  dt2 = Date(int(arr2[1]), int(arr2[0]), int(arr2[2]))   #checkin date
  dt3 = Date(int(arr3[1]), int(arr3[0]), int(arr3[2]))   #checkout date
  diff1.append(getDifference(dt1, dt2))   #day difference between booking and checkin
  diff2.append(getDifference(dt2, dt3))   #day difference between checkin and checkout

In [ ]:
np_month = np.array(month).reshape(len(booking),1)   #reshaping np obj
np_diff1 = np.array(diff1).reshape(len(booking),1)   #reshaping np obj
np_diff2 = np.array(diff2).reshape(len(booking),1)   #reshaping np obj

In [ ]:
z = np.append(z, np_month, axis=1)    #append new column | month of checkin -> to capture seasonal variations
z = np.append(z,np_diff1, axis=1)     #append new column  | duration of prior booking before checkin
z = np.append(z, np_diff2, axis=1)    #append new column | duration of hotel stay i.e. difference between checkin vs checkout

In [ ]:
z = np.delete(z,7,axis=1)       #delete checkin column
z = np.delete(z,7,axis=1)       #delete checkout column
z = np.delete(z,7,axis=1)       #delete bookin column

In [ ]:
#ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(), [0,2,3,4,5,6,10,11,12,13,14,15,16,19])], remainder='passthrough')
z = ct.transform(z)      #hot encode lable the categorical data

In [ ]:
z[:, 50:] = sc.transform(z[:, 50:])       #use the same scaler

In [ ]:
y_pred = classifier.predict(z)            #predict for the test set using the trained classifier

In [ ]:
import csv                                  #write out to a csv file
with open('results.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    results = []
    for i in y_pred:
      writer.writerow([i])
      if i not in results:
        results.append(i)
        print (i)

1
2
3
